In [ ]:
# ==============================================
# 1. Install dependencies
# ==============================================
!pip install langchain langchain-community faiss-cpu sentence-transformers transformers accelerate pypdf

# ==============================================
# 2. Imports
# ==============================================
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from google.colab import files

# ==============================================
# 3. Upload PDF
# ==============================================
uploaded = files.upload()
pdf_path = list(uploaded.keys())[0]
print(f"Uploaded PDF: {pdf_path}")

# ==============================================
# 4. Load & Split PDF
# ==============================================
loader = PyPDFLoader(pdf_path)
documents = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = splitter.split_documents(documents)

print(f"Total chunks created: {len(docs)}")

# ==============================================
# 5. Embeddings + Vectorstore
# ==============================================
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db = FAISS.from_documents(docs, embeddings)

# ==============================================
# 6. Load Free HuggingFace Model (FLAN-T5)
# ==============================================
model_id = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

pipe = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256
)

llm = HuggingFacePipeline(pipeline=pipe)

# ==============================================
# 7. Custom Prompt Template
# ==============================================
template = """You are an expert assistant. Answer clearly and concisely using the given context
and the chat history if relevant. If the answer is not in the context, say "I don’t know".

Chat history:
{chat_history}

Context:
{context}

Question:
{question}

Answer:"""

custom_prompt = PromptTemplate.from_template(template)

# ==============================================
# 8. Add Memory + Conversational Retrieval Chain
# ==============================================
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

qa = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=db.as_retriever(search_kwargs={"k": 3}),
    memory=memory,
    combine_docs_chain_kwargs={"prompt": custom_prompt}
)

# ==============================================
# 9. Interactive Q&A
# ==============================================
print("✅ Setup complete! Now you can chat with your PDF.\n")

while True:
    query = input("Ask a question (or type 'exit'): ")
    if query.lower() == "exit":
        break
    result = qa.invoke({"question": query})
    print(f"\n🤖 Answer: {result['answer']}\n")


Saving History of Bhopal.pdf to History of Bhopal (4).pdf
Uploaded PDF: History of Bhopal (4).pdf
Total chunks created: 58


Device set to use cpu
/tmp/ipython-input-425466988.py:82: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


✅ Setup complete! Now you can chat with your PDF.

Ask a question (or type 'exit'): who is gohar begum ?

🤖 Answer: She was the widow of Nawab Nazar Mohammad Khan, who died suddenly, leaving behind an infant daughter, Sikandar.

Ask a question (or type 'exit'): how long did she rule for ?

🤖 Answer: In 1926, in a move that surprised many, she abdicated the throne in favor of her son, Hamidullah Khan, bringing to a close the extraordinary century of female rule that had for the survival of her state.

Ask a question (or type 'exit'): who ?

🤖 Answer: Gohar Begum

Ask a question (or type 'exit'): i thought sultan jahan was the last begum

🤖 Answer: Nawab Faiz Mohammad Khan

Ask a question (or type 'exit'): exit
